In [233]:
# Uses beautifulsoup4, urllib, json
# Before every commit, do "git pull --rebase"

'''first, get track ids from genre list

create dictionary with key( genre) and value( list of track ids )

for each track, get id, user_id, genre, tag_list
    for the user in the track:
        id, uri, permalink_url, country'''

from bs4 import BeautifulSoup
import urllib.request
import json
import csv as csv

page_url = "http://www.soundcloud.com/charts"
page_html = urllib.request.urlopen( page_url ).read()

page_beautifulsoup_object = BeautifulSoup( page_html, "html.parser" )

page_html_file = open( "html_saved_prettified.html", "w" )
page_html_file.write( page_beautifulsoup_object.body.div.find_all( "noscript" )[ 1 ].section.section.find_all( "article" )[ 1 ].ul.prettify() )

page_beautifulsoup_genres_array = page_beautifulsoup_object.body.div.find_all( "noscript" )[ 1 ].section.section.find_all( "article" )[ 1 ].ul

genre_song_list_length_limit = 100

api_url_chunk_1 = "https://api-v2.soundcloud.com/charts?kind=top&genre=soundcloud%3Agenres%3A"
api_url_chunk_2 = "&client_id=02gUJC0hH2ct1EGOcYXQIzRFU91c72Ea&limit=" + str( genre_song_list_length_limit ) + "&offset=0&linked_partitioning=1&app_version=1476434302"

genre_list = []

iterator = 0
for value in page_beautifulsoup_genres_array:
    if iterator > 3 and iterator % 2 == 1:
        # Reverse search through the HTML to find the first and last indices of the genre name
        genre_string_start_index = repr( value ).rfind( "=" ) + 1
        genre_string_end_index = repr( value ).rfind( "\"" )
        genre_string = repr( value )[ genre_string_start_index : genre_string_end_index ]
        genre_list.append( genre_string )
    iterator = iterator + 1

f = open("tracks1.csv","w")
f1 = csv.writer(f)
for genre in genre_list:
    json_data = urllib.request.urlopen( api_url_chunk_1 + genre + api_url_chunk_2 ).read().decode( "utf-8" )
    json_object = json.loads( json_data ).get( "collection" )
    
    for song_dict in json_object:
        #f1.writerow([song_dict["track"]["user_id"], song_dict["track"]["user"]["username"]])
        f1.writerow([song_dict["track"]["id"], song_dict["track"]["user_id"],song_dict["track"]["title"],genre,song_dict["track"]["playback_count"],song_dict["track"]["likes_count"]])

f.close()


In [235]:
import pandas as pd
df = pd.read_csv("tracks1.csv")
df.columns = ["track_id", "user_id", "title", "genre", "playback_count", "likes"]

In [167]:
df.drop_duplicates(inplace=True)

,track_id,user_id,title,genre,playback_count,likes
0,272105509,48084634,xxxtentacion - YuNg BrAtZ Prod. STAIN,alternativerock,1271007.0,28174.0
1,270817339,10494998,Stole Your Luv ++ Prod. Lyle LeDuff,alternativerock,4723663.0,94837.0
2,263994790,99685284,Hollywood Dreams / Comedown (Prod. by FKi & L...,alternativerock,3007916.0,50156.0
3,274770566,505203,Playboi Carti - Pump Fake! (Freestyle),alternativerock,1050026.0,28004.0
4,229528669,3932070,Emperor's New Clothes,alternativerock,5253830.0,79319.0


In [230]:
df.to_csv("tracks_and_artists.csv")

In [172]:
df.name.str.len().max()

50

In [174]:
df.shape

(2257, 2)

In [185]:
import string
printable  = set(string.printable)

In [204]:
df[df["name"].isin(printable)]

,user_id,name


In [202]:
df[["user_id", "name"]]

,user_id,name
0,48084634,XXXTENTACION
1,10494998,LIL UZI VERT
2,99685284,Post Malone
3,505203,ETHEREAL
4,3932070,Panic! At The Disco
5,250567331,Lil Yatchy ⛵️
6,1377057,Interscope Records
9,180986905,Brain Bakery Mag
12,67377000,SKI MASK THE SLUMP GOD
13,1856723,Fitz and The Tantrums


In [206]:
a = lambda x: x.encode('ascii', 'ignore')

In [237]:
df["title"] = df["title"].apply(a)

In [238]:
df

,track_id,user_id,title,genre,playback_count,likes
0,272105509,48084634,b'xxxtentacion - YuNg BrAtZ Prod. STAIN',alternativerock,1271007.0,28174.0
1,270817339,10494998,b'Stole Your Luv ++ Prod. Lyle LeDuff',alternativerock,4723663.0,94837.0
2,263994790,99685284,b'Hollywood Dreams / Comedown (Prod. by FKi &...,alternativerock,3007916.0,50156.0
3,274770566,505203,b'Playboi Carti - Pump Fake! (Freestyle)',alternativerock,1050026.0,28004.0
4,229528669,3932070,"b""Emperor's New Clothes""",alternativerock,5253830.0,79319.0
5,283695246,250567331,b'Lil Yachty - Tip Toe',alternativerock,110297.0,3886.0
6,36650065,1377057,b'Imagine Dragons - Demons',alternativerock,10514587.0,186875.0
7,260906332,48084634,b'KING',alternativerock,560208.0,12440.0
8,226032182,3932070,b'Victorious',alternativerock,4363867.0,72345.0
9,261371411,180986905,b'xxxtentacion - Valentine (Prod. xxx & Mikey ...,alternativerock,474623.0,8303.0


In [229]:
pd.read_csv("tracks_and_artists.csv")

OSError: File b'tracks_and_artists.csv' does not exist